In [2]:
!pip install huggingface_hub transformers accelerate datasets sentencepiece evaluate sacrebleu --quiet

In [3]:
#!pip install accelerate -U --quiet

In [4]:
#!pip install transformers[torch] --quiet
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [1]:
import transformers

print(transformers.__version__)

4.31.0


In [2]:
from huggingface_hub import notebook_login
from datasets import load_dataset , load_metric
from datasets import Dataset
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

notebook_login()

In [3]:
# Load the dataset
data = load_dataset("cfilt/iitb-english-hindi")
metric = load_metric("sacrebleu")

Repo card metadata block was not found. Setting CardData to empty.
/var/tmp/ipykernel_7388/2907098826.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [5]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(data["train"])

,translation
0,"{'en': 'They are: - large reservoir of scientific human resource, that is, a strong pool of scientists and engineers; cost effective manufacturing capabilities; number of national research laboratories employing thousands of scientists; centers of academic excellence in biosciences; several medical colleges, educational and training institutes offering degrees and diplomas in biotechnology, bio - informatics and biological sciences; vibrant drugs and pharmaceutical industry; as well as fast developing clinical capabilities.', 'hi': 'वे हैं: वैज्ञानिक मानव संसाधन का विशाल भंडार अर्थात वैज्ञानिकों और अभियंताओं का एक मजबूत समूह, किफायती विनिर्माण क्षमताएं, अनेक राष्ट्रीय अनुसंधान प्रयोगशालाएं, जिसमें हजारों वैज्ञानिकों को रोजगार मिला हुआ है, जैव विज्ञान में अकादमी उत्कृष्टता के केन्द्र, अनेकानेक मेडिकल कॉलेज, शैक्षिक और प्रशिक्षण संस्थान, जो जैव प्रौद्योगिकी में डिग्री और डिप्लोमा प्रदान करते हैं, जैव-सूचना विज्ञान और जीव विज्ञानी विज्ञान, असरदार औषध और भेषज उद्योग, तथा तेजी से विकसित होती उपचारात्मक क्षमताएं। '}"
1,"{'en': 'Hast thou seen if he denieth (Allah 's guidance) and is froward?', 'hi': 'तुम्हारा क्या विचार है? यदि उस (रोकनेवाले) ने झुठलाया और मुँह मोड़ा (तो उसके बुरा होने में क्या संदेह है)-'}"
2,"{'en': 'In all things We have created pairs so that you may take admonition.', 'hi': 'और हमने हर चीज़ के जोड़े बनाए, ताकि तुम ध्यान दो'}"
3,"{'en': 'Is not Allah the most just of judges?', 'hi': 'क्या ख़ुदा सबसे बड़ा हाकिम नहीं है (हाँ ज़रूर है) '}"
4,"{'en': 'Rashtrakuta Dynasty', 'hi': 'राष्ट्रकूट राजवंश '}"


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/mt5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
source_lang = "en"
target_lang = "hi"
prefix = "Translate English to Hindi: "

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=250, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=250, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(data['train'][:3])

In [ ]:
tokenized_bilingual_pairs = data.map(preprocess_function, batched=True)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [16]:
import torch
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
torch.cuda.empty_cache()
output_dir = 'bilingual_translation'

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True, # Uncomment if use GPU
    push_to_hub=True,
    logging_steps=25

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bilingual_pairs["train"],
    eval_dataset=tokenized_bilingual_pairs["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/home/jupyter/bilingual_translation is already a clone of https://huggingface.co/amitkumarjaiswal/bilingual_translation. Make sure you pull the latest changes with `repo.git_pull()`.


In [18]:
import gc

gc.collect()

0

In [19]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 6.53 GiB (GPU 0; 39.41 GiB total capacity; 30.57 GiB already allocated; 4.86 GiB free; 32.82 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF